In [2]:
# https://medium.com/@jebaseelanravi96/machine-learning-iris-classification-33aa18a4a983


import numpy as np
import pandas as pd

input_file = "iris_orig.csv"

iris_df = pd.read_csv(input_file, header = 0)
iris_df.head()

# I will likely pollute the data a bit and have them do some kind of data cleaning, right now its clean though
# I also want to introduce a "categorical" feature like color, just to force them to 
# encode it / do a tiny bit of data transforming
# data.isnull()

feature_cols = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm','PetalWidthCm']



x = iris_df.loc[:, feature_cols]
x.shape
y = iris_df.Species

In [3]:
x

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [4]:


from sklearn.model_selection import GridSearchCV,train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.5)

from sklearn import tree


classifier=tree.DecisionTreeClassifier()


classifier.fit(x_train,y_train)
predictions=classifier.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions))

0.9333333333333333


In [5]:
# Now let's encourage an optimization workflow here, somehow can we coach them to track this via jobs?
# I found some pointers on optimizing a decision tree here 
# https://ai.plainenglish.io/hyperparameter-tuning-of-decision-tree-classifier-using-gridsearchcv-2a6ebcaffeda



param_dict = {
    "criterion":['gini','entropy'],
    "max_depth":range(1,10),
    "min_samples_split":range(2,10),
    "min_samples_leaf":range(1,5),
}
# (imported  GridSearchCV already in previous cell)
grid = GridSearchCV(classifier,
                       param_grid=param_dict,
                       cv=10,
                       verbose=1,
                       n_jobs=-1)

# note this kind of seems like its working but then there's an error? I can't tell.
grid.fit(x_train,y_train)

Fitting 10 folds for each of 576 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 5128 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed:    4.5s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                    

In [13]:
#print the best parameters
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 4,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [14]:
#new accuracy score after grid search
predictions_grid = grid.predict(x_test)
print(accuracy_score(y_test,predictions_grid))

0.96


In [15]:
#fit model on entire dataset
grid.fit(x,y)

Fitting 10 folds for each of 576 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 4624 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed:    4.5s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                    

In [16]:
#export the trained model as a pickle file for later use
import pickle
with open("model.pkl", "wb") as f:
      pickle.dump(grid, f)